# Example: Let's Explore Binomial Lattice Model Trade Exit Rules
In this example, we will test the binomial lattice model trade exit rules we developed in lecture.
In particular, we will use a binomial lattice model to compute the cumulative probability of achieving at least a target fractional return (ROI) of $r_{\star}$ (or alternatively, a target fractional loss of $-r_{\star}$) over a holding period of $T=i\Delta{t}$.

> __Leaning Objectives__
> 
> Fill me in later!

This sounds cool, so let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

>__Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

### Data
We gathered daily open-high-low-close (OHLC) data for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2014` until `12-31-2024`, along with data for a few exchange-traded funds and volatility products during that time period. 

Let's load the `original_dataset::DataFrame` by calling [the `MyTrainingMarketDataSet()` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/data/#VLQuantitativeFinancePackage.MyTrainingMarketDataSet) and remove firms that do not have the maximum number of trading days. The cleaned dataset $\mathcal{D}$ will be stored in the `dataset` variable.

In [2]:
original_dataset = MyTrainingMarketDataSet() |> x-> x["dataset"];

Not all tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquisition or delisting events. Let's collect only those tickers with the maximum number of trading days.

First, let's compute the number of records for a firm that we know has the maximum value, e.g., `AAPL`, and save that value in the `maximum_number_trading_days::Int64` variable:

In [3]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow # nrow? (check out: DataFrames.jl)

2767

Now, let's iterate through our data and collect only tickers with `maximum_number_trading_days` records. We'll save that data in the `dataset::Dict{String,DataFrame}` variable:

In [4]:
dataset = let

    # initialize -
    dataset = Dict{String, DataFrame}();

    # iterate through the dictionary; we can't guarantee a particular order
    for (ticker, data) ∈ original_dataset  # we get each (K, V) pair!
        if (nrow(data) == maximum_number_trading_days) # what is this doing?
            dataset[ticker] = data;
        end
    end
    dataset; # return
end;

How many firms do we have with the full number of trading days? Let's use [the `length(...)` method](https://docs.julialang.org/en/v1/base/collections/#Base.length) — notice this works for dictionaries, in addition to arrays, sets, and other collections.

In [5]:
length(dataset) # tells us how many keys are in the dictionary (how many firms in our dataset?)

424

Finally, let's get a list of the firms in our cleaned dataset and sort them alphabetically. We store the sorted firm ticker symbols in the `list_of_tickers::Array{String,1}` variable.

In [6]:
list_of_tickers = keys(dataset) |> collect |> sort # list of firm "ticker" symbols in alphabetical order

424-element Vector{String}:
 "A"
 "AAL"
 "AAP"
 "AAPL"
 "ABBV"
 "ABT"
 "ACN"
 "ADBE"
 "ADI"
 "ADM"
 ⋮
 "WYNN"
 "XEL"
 "XOM"
 "XRAY"
 "XYL"
 "YUM"
 "ZBRA"
 "ZION"
 "ZTS"

### Constants
Finally, let's set some constants we'll use later in this notebook. The comments describe the constants, their units, and permissible values.

In [7]:
TSIM = 63; # number of trading days to simulate
Δt = (1.0/252); # step size: 1 trading day in units of years
r̄ = 0.05; # risk-free rate (annualized)

___

## Task 1: Estimate lattice parameters from historical data
After validating our lattice implementation, it's time to utilize historical data to compute a share price prediction. To create a binomial lattice model for future share prices, we must estimate three critical parameters: $p$, $u$, and $d$.

> __Parameter Definitions__
>
>* The $p$ parameter represents the probability of a share price increase or an `up` move between two consecutive periods $j\rightarrow{j+1}$. Since a binary lattice model only allows `up` and `down` moves, the probability of a `down` move is $1-p$.
>* The $u$ parameter represents the magnitude of an `up` move. If $S_{j}$ denotes the share price in period $j$, and $S_{j+1}$ is the share price in the next period, then an `up` move results in $S_{j+1} = u\cdot{S}_{j}$.
>* The $d$ parameter represents the magnitude of a `down` move. If $S_{j}$ denotes the share price in period $j$, and $S_{j+1}$ is the share price in the next period, then a `down` move results in $S_{j+1} = d\cdot{S}_{j}$.

To start, let's select a firm from the dataset to explore.

In [8]:
# random_firm_ticker = rand(list_of_tickers);
random_firm_ticker = "AAPL"
random_firm_index = findfirst(x-> x == random_firm_ticker, list_of_tickers);
random_firm_data = dataset[random_firm_ticker];

### Estimate the u, d, and probability p parameters from the data
Let's estimate the up $u$ and down $d$ factors and the probability $p$ from historical data.

__Initialize__: Given the growth rate sequence $\{\mu_{2},\mu_{3},\dots,\mu_{T}\}$ for firm $(i)$ (we omit the superscript $i$ for simplicity) and a time step $\Delta{t} > 0$ (units: years), initialize the up factors collection $U = \emptyset$ and down factors collection $D = \emptyset$.

1. For $t = 2,3,\dots,T$, __do__:
    - If $\mu_{t} > 0$, then update $U \leftarrow U \cup \{e^{\mu_{t}\Delta{t}}\}$, where $U$ is a collection of up factors.
    - If $\mu_{t} < 0$, then update $D \leftarrow D \cup \{e^{\mu_{t}\Delta{t}}\}$, where $D$ is a collection of down factors.
    - If $\mu_{t} = 0$, skip (no price change).
2. Compute the up factor $u$ as the mean of the up factors collection $U$: $u = \frac{1}{|U|} \sum_{v \in U} v$.
3. Compute the down factor $d$ as the mean of the down factors collection $D$: $d = \frac{1}{|D|} \sum_{v \in D} v$.
4. Compute the probability $p$ as the fraction of up movements: $p = \frac{|U|}{|U| + |D|}$.

We use daily data; thus, the natural time frame between $S_{j-1}$ and $S_{j}$ is a single day. However, it is more convenient to use an annualized value for the $\mu$ parameter; thus, we let $\Delta{t} = 1/252$, i.e., the fraction of a year that corresponds to a single trading day.

The [`log_growth_matrix(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.log_growth_matrix) takes the cleaned dataset $\mathcal{D}$, which contains $T$ days of data for each firm, a list of firms $\mathcal{L}$, and computes the growth rate values for each firm as a function of time. 

The data is returned as a $(T - 1)\times\dim\mathcal{L}$ array (time on the rows, firm $i$ on the columns). We store the data in the `log_growth_array::Array{Float64,1}` variable:

In [9]:
log_growth_array = log_growth_matrix(dataset, random_firm_ticker) # array holding growth rate time series

2766-element Vector{Float64}:
 -1.9954223478275546
  0.15124876922831154
  0.9224750663536962
 -1.4261203338251638
 -3.0008957287125435
  1.5049147899343094
  2.9156076155145367
  6.502627170431676
 -1.2561327051214097
 -4.024741418592851
  ⋮
 -2.325529514930973
 -0.7888121514647656
  3.27667430452004
  1.6089944655721826
  2.6547619721872575
  1.4556917537856608
 -3.6286559032528003
 -2.934268163874707
 -1.5398398549769603

Using the `log_growth_array::Array{Float64,1}`, we compute the expected magnitude of an up move $\bar{u}$, the expected magnitude of a down move $\bar{d}$, and the estimated probability $\bar{p}$ of an up move by passing the return data to [a `RealWorldBinomialProbabilityMeasure` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.RealWorldBinomialProbabilityMeasure), which returns the $(u,d,p)$ tuple.

> __Syntax sugar__: Behind the scenes, this call points to a private function that computes $\bar{u}$, $\bar{d}$, and $\bar{p}$ by analyzing the growth rate time series data in the `log_growth_array::Array{Float64,1}` variable. It looks like we are using a type as a function name, but we are not. This is a neat Julia trick, right?

What values did we obtain for our random firm?

In [10]:
(ū,d̄,p̄) = let

    # initialize -
    u = nothing; 
    d = nothing; 
    p = nothing; 

    # TODO: Uncomment the line below to compute the RWPM parameters for the randomly selected firm
    (u,d,p) = (RealWorldBinomialProbabilityMeasure())(log_growth_array; Δt = Δt)
    
    (u,d,p); # return
end;

Make a table for the binomial lattice parameters:

In [11]:
let

    # initialize -
    df = DataFrame();

    row_df = (
        ticker = random_firm_ticker,
        upfactor = ū,
        downfactor = d̄,
        probability = p̄,
    );
    push!(df, row_df);

      # display the table -
    pretty_table(df, backend = :text,
         table_format = TextTableFormat(borders = text_table_borders__compact));    
    
end

 -------- ---------- ------------ -------------
  ticker   upfactor   downfactor   probability 
  String    Float64      Float64       Float64 
 -------- ---------- ------------ -------------
    AAPL    1.01059     0.989301      0.550976
 -------- ---------- ------------ -------------


### Build binomial lattice model using historical parameters
Let's construct an instance of [the `MyBinomialEquityPriceTree` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.MyBinomialEquityPriceTree) using the estimated values for the parameters ($\bar{u}$, $\bar{d}$, $\bar{p}$) from above. 

This enables us to calculate the prices and probabilities in the tree. We store the populated model in the variable `random_test_model` for future use.

First, specify the `start_index::Int64` as the trading day index in the dataset, which will serve as the tree's starting point or `L = 0`. For reproducibility, let's hard-code this value as `start_index = 1187`, which corresponds to the trading day `2018-09-19` for our dataset.

In [12]:
start_index = 1187; # DO NOT CHANGE hardcoded start-time for reproducibility
stop_index = start_index + TSIM # TSIM defines the number of trading days to simulate
println("Visualize Firm-$(random_firm_index) between trading days ($(start_index) -> $(stop_index))")

Visualize Firm-4 between trading days (1187 -> 1250)


Next, let's build and populate the binomial lattice model using the estimated parameters and the initial share price $S_{0}$, which we set as the volume-weighted average price (VWAP) on the `start_index` trading day.

We save the populated model in the `random_test_model::MyBinomialEquityPriceTree` variable:

In [13]:
random_test_model = let

    # initialize -
    model = nothing;
    Sₒ = random_firm_data[start_index, :volume_weighted_average_price]; # set the initial share price
    
    
    # TODO: Uncomment the line below and re-run this cell to build and populate the binomial equity price tree model
    # TODO: using the (ū, d̄, p̄) values computed above (real-world parameters)
    model = build(MyBinomialEquityPriceTree, (
        u = ū, d = d̄, p = p̄)) |> (model -> populate(model, Sₒ = Sₒ, h = TSIM));
    
    
    model; # return
end;

## Task 2: Compute cumulative probability of achieving target return
In this task, we'll compute the probability of achieving at least a target fractional return $r_{\star}$ over a holding period of $T = i\Delta{t}$. This probability is denoted as $P(r > r_{\star})$ is computed as:
$$
\boxed{
\mathbb P\!\left(\frac{\texttt{NPV}(\bar r,T)}{n_{o}S_{0}}>r_\star\right)
=\sum_{k=k_{\min}}^{i}\binom{i}{k}p^{k}(1-p)^{\,i-k}
=1-\sum_{k=0}^{k_{\min}-1}\binom{i}{k}p^{k}(1-p)^{\,i-k}.
}
$$
where $k_{\min}$ is the minimum number of `up` moves required to achieve at least the target return $r_{\star}$ over the holding period $T = i\Delta{t}$, and is computed as:
$$
\boxed{
k_{\min}
=
\left\lceil
\frac{\ln(1+r_\star)+\bar r\,i\,\Delta t - i\ln d}{\ln(u/d)}
\right\rceil
}
\quad\text{(clamp to $[0,i]$).}
$$

Let's set a target fractional return over the holding period. We store this value in the `f₊::Float64` variable:

In [14]:
f₊ = -0.05; # target return over the simulation horizon

Next, we'll compute the minimum number of `up` moves $k_{\min}$ required to achieve at least the target return $r_{\star}$ over the holding period $T = i\Delta{t}$. We store this value in the `k₊::Int64` variable:

In [15]:
kmin = let 

    # initialize -
    i = TSIM;
    N = log(1.0 + f₊) + r̄*i*Δt - i*log(d̄);
    D = log(ū) - log(d̄);
    k = floor(Int, N/D)+1; # round to nearest integer
    
    min(max(k,0),i); # clamped return
end;

In [16]:
println("Minimum `up` moves to achieve at least the target return $(f₊*100)% over period T = $(TSIM*Δt*252) trading days is kmin = $(kmin)")

Minimum `up` moves to achieve at least the target return -5.0% over period T = 63.0 trading days is kmin = 31


The probability of achieving at least the target return $r_{\star}$ over the holding period $T = i\Delta{t}$ is then given by (stored in the `test_probability::Float64` variable):

In [17]:
test_probability = let 
    
    # initialize -
    i = TSIM;

    P = 0.0;
    for k ∈ kmin:i
        a = binomial(i, k);
        b = p̄^k;
        c = (1.0 - p̄)^(i - k);
        P+= a*b*c; # accumulate
    end 

    P;
end;

In [18]:
println("Probability of achieving at least the target return $(f₊*100)% over the holding period T = $(TSIM*Δt*252) trading days is P = $(round(test_probability*100, digits=4))%")

Probability of achieving at least the target return -5.0% over the holding period T = 63.0 trading days is P = 85.6872%


Fill me in later!

In [ ]:
P = let

    # initialize -
    i = TSIM;
    number_of_target_returns = 9;
    f = range(-0.05, stop = 0.15, length = number_of_target_returns) |> collect; # target return values;
    R = Array{Float64,2}(undef, number_of_target_returns, 2); # array to hold cumulative probabilities
    
    # main loop - iterate through each target return, compute k₊, then compute cumulative probability
    for r ∈ eachindex(f)
        
        # compute k₊ for this target return
        N = log(1.0 + f[r]) + r̄*i*Δt - i*log(d̄);
        D = log(ū) - log(d̄);
        k = floor(Int, N/D) + 1; # floor to nearest integer + 1
        kmin = min(max(k,0),i); # clamp to [0,i]
        
        # compute cumulative probability of achieving this target return
        P = 0.0;
        for k ∈ kmin:i
            a = binomial(i, k);
            b = p̄^k;
            c = (1.0 - p̄)^(i - k);
            P+= a*b*c; # accumulate
        end 
        R[r,1] = f[r]; # store
        R[r,2] = P; # store
    end

    R;
end;

Let's make a table of the results using [the `pretty_table(...)` function exported by the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl):

In [22]:
let

    # initialize -
    df = DataFrame();

    for r ∈ eachrow(P)
        row_df = (
            target_return = r[1],
            P = r[2],
            P̄ = 1.0 - r[2],
        );
        push!(df, row_df);
    end

      # display the table -
    pretty_table(df, backend = :text,
         table_format = TextTableFormat(borders = text_table_borders__compact));    
end

 --------------- ---------- ----------
  target_return          P          P̄ 
        Float64    Float64    Float64 
 --------------- ---------- ----------
          -0.05   0.856872   0.143128
         -0.025   0.792339   0.207661
            0.0   0.713154   0.286846
          0.025   0.621878   0.378122
           0.05   0.523054   0.476946
          0.075    0.42258    0.57742
            0.1    0.32669    0.67331
          0.125   0.240828   0.759172
           0.15   0.168742   0.831258
 --------------- ---------- ----------


## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy, is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.